# Celery worker setup

1) add <font color='yellow'>celery.py</font> to the main app and add these lines
<font color='deepskyblue'>
> from celery import Celery

> os.environ.setdefault("DJANGO_SETTINGS_MODULE", "celerytst.settings")

> app.config_from_object("django.conf:settings", namespace="CELERY")

> app.autodiscover_tasks()
</font>
2) add this to setting.py  
<font color='deepskyblue'>
> CELERY_BROKER_URL = os.getenv("CELERY_BROKER_URL", "redis://redis:6379/0")
</font>
3) add <font color='yellow'>tasks.py</font> to your app and use <font color='yellow'>@shared_task</font> decorator to define it as a task   

# Independed Celery worker

1) make a new directory next to yor project dir 
2) add a <font color='yellow'>celerytask.py</font> and <font color='yellow'>celeryconfig.py</font> to it
3) add <font color='yellow'>Dockerfile</font> and <font color='yellow'>requirement.txt</font> if you want to build it seperately
4) in <font color='yellow'>celerytask.py</font> make a new celery instance and choose celeryconfig as a setting file 
<font color='deepskyblue'>
    > app = Celery('mytask')
    
    > app.config_from_object('celeryconfig')
    
    > @app.task

    > def your_task():
    >   return
</font>

5) add congifs to the celeryconfig.py
<font color='deepskyblue'>
    > broker_url = 'redis://redis:6379/0'
    > result_backend = 'redis://redis:6379/0'
</font>

6) add a service using this directory to the docker compose file

## Celery settings

In [ ]:
'''-------------- celery@44b015f2d5cf v5.5.1 (immunity)
 --- ***** ----- 
 -- ******* ---- Linux-6.10.14-linuxkit-x86_64-with-glibc2.36 2025-04-12 06:45:38
 --*** --- * --- 
 --** ---------- [config]
 --** ---------- .> app:         mytask:0x7f0ddfae1c00
 --** ---------- .> transport:   redis://redis:6379/0   -----------------------------------> these two lines must be redis
 --** ---------- .> results:     redis://redis:6379/0   -----------------------------------> these two lines must be redis
 --*** --- * --- .> concurrency: 16 (prefork)
 -- ******* ---- .> task events: OFF (enable -E to monitor tasks in this worker)
 --- ***** ----- 
  -------------- [queues]
                 .> celery           exchange=celery(direct) key=celery
                 
 
 [tasks]
   . celerytask.your_task  ----------------------------> here your detected tasks will be shown                     '''

# Workers routing

we can route our tasks to diffrent workers and routing could be based on load balance or task types

passing to queue:

1) we use <font color='yellow'>-Q <queuename> flag</font> to set a worker to watch a individuaqueue  <font color='yellow'>celery -A <#filename> worker --loglevel=info -Qqueue2</font> in order to pass a task to a spicifiedqueuee this command in celery.py 
    <font color='deepskyblue'>
    >app.conf.task_routes = {'newapp.tasks.task1': {'queue':'queue1'}}
    
    >app.conf.task_routes = {'<#task location>': {'queue':'<queuename>'},}
    </font> 
3) if there is a independed worker we have to creat a <font color='yellow'>task.py</font> file and write tasks in there and import task using this command
    <font color='deepskyblue'>
    >app.conf.imports = ('newapp.tasks')

    >app.conf.imports = ('<#task location>')
    
    >app.autodiscover_tasks()
    </font>



# Tasks priority

in celery prioritazing could be done in two ways:
1) set priority to a task
2) set priority to a queue and put task in them based on your priority

Redis does not support first method and we should use RabitMQ to do that

### set priority to a queue

1) first set multiple queus for worker in docker-compose 
> celery -A celerytst worker --loglevel=info -Q queue1,queue2,queue3
remember that when passing -Q queus are sorted based on their priority meaning queue1 > queue2 > queue3
2) now set each task to a queue using app.conf.task_routes
app.conf.task_routes = {
    'newapp.tasks.task1': {'queue':'queue1'}, 
    'newapp.tasks.task2': {'queue':'queue2'},
    'newapp.tasks.task3': {'queue':'queue3'},
    }
3) now task1 is prioritized on task2 and task2 on task3

# Task groups

we use groups to run multiple tasks together in parallel
1) first we have to make a group out of tasks with this command
> from celery import group
> task_group = group(task1.s(), task2.s(),task3.s())
2) to execute the task group use
> task_group.apply_async()
3) now task1,2,3 are executed together
